## Prevendo o preço de casas

Base de dados simplicada do desafio [House Prices](https://www.kaggle.com/c/house-prices-advanced-regression-techniques) do Kaggle.

Vamos tentar prever o preço de casas baseado no seu tamanho, ano de construção e capacidade de carros na garagem.

Durante a modelagem, poderemos observar como o MLFlow vai nos ajudar a acompanhar os resultados do experimento e tornar o nosso projeto mais maduro.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/processed/casas.csv')
df.head()

,tamanho,ano,garagem,preco
0,159.0,2003,2,208500
1,117.0,1976,2,181500
2,166.0,2001,2,223500
3,160.0,1915,3,140000
4,204.0,2000,3,250000


In [3]:
X = df.drop('preco', axis=1)
y = df['preco']

## Treino e teste

Como de costume, separamos a nossa base em treino e teste antes de começar o treino dos modelos de Machine Learning.

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [6]:
X_train.head()

,tamanho,ano,garagem
135,156.0,1970,2
1452,100.0,2005,2
762,144.0,2009,2
932,177.0,2006,3
435,154.0,1996,2


## Regressão Linear

Primeiro vamos ajustar uma regressão linear ao nosso modelo. Lembrando que vamos usar a regressão linear e outras técnicas de forma bem simples, uma vez que o objetivo do nosso experimento não é uma ótima performance preditiva mas sim como o MLFlow pode ser útil para a fase de experimentação.

In [7]:
from sklearn.linear_model import LinearRegression

In [8]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [9]:
lr.predict([[120,2001,2]])

array([191436.53503831])

In [10]:
from sklearn.metrics import mean_squared_error, r2_score

In [11]:
lr_predicted = lr.predict(X_test)

In [12]:
import math

mse = mean_squared_error(y_test,lr_predicted)
rmse = math.sqrt(mse)
r2 = r2_score(y_test,lr_predicted)

In [13]:
print(f'mse: {mse}, rmse: {rmse}, r2: {r2}')

mse: 2078666917.9289918, rmse: 45592.399782518485, r2: 0.7021153642898047


## XGBoost

Uma abordagem comum durante o processo de experimentação é testar outras técnicas e também fazer a tunagem dos parâmetros. Para ver se o nosso modelo pode ser melhorado, vamos testar o ajuste de um modelo com o XGBoost.

In [14]:
from xgboost import XGBRegressor

In [15]:
xgb = XGBRegressor(random_state=42)
xgb.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [16]:
xgb_predicted = xgb.predict(X_test)
mse = mean_squared_error(y_test,xgb_predicted)
rmse = math.sqrt(mse)
r2 = r2_score(y_test,xgb_predicted)

In [17]:
print(f'mse: {mse}, rmse: {rmse}, r2: {r2}')

mse: 1550816053.129738, rmse: 39380.40189141977, r2: 0.7777593557411607


### Testando novos parâmetros

A fim the otimizar a performance preditiva do modelo de machine learning é comum testarmos diferentes combinações de parâmetros para cada técnica escolhida. Vamos treinar um novo modelo com diferentes parâmetros e ver se conseguimos melhorar a performance do nosso modelo.

In [18]:
xgb = XGBRegressor(learning_rate=0.2,n_estimators=50,random_state=42)
xgb.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=50, n_jobs=8, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [19]:
xgb_predicted = xgb.predict(X_test)
mse = mean_squared_error(y_test,xgb_predicted)
rmse = math.sqrt(mse)
r2 = r2_score(y_test,xgb_predicted)
print(f'mse: {mse}, rmse: {rmse}, r2: {r2}')

mse: 1386727460.1346002, rmse: 37238.789724353286, r2: 0.8012741720529797


## Usando o MLFlow para fazer o tracking dos experimentos

Podemos notar que conforme vamos evoluindo no nosso notebook, testando técnicas e combinações de parâmetros, evoluimos no entendimento e construção do modelo de machine learning. No entanto, conforme o projeto cresce aumenta-se a complexidade e o risco de rastrear quais diferentes técnicas trouxeram um resultado, e caso se escolha por alguma configuração em específica, a reprodução desses resultados pode ser complexa, uma vez que precisamos garantir que o estado de todas as variáveis é o mesmo. Essa complexidade aumenta ainda mais quando precisamos compartilhar os experimentos e resultados com colegas de times, onde reproduzir os resultados em diferentes máquinas e ambientes pode ser ainda mais difícil.

Vamos utilizar o MLFlow tracking para rastrear os resultados dos experimentos e conseguir reproduzir os modelos sem dor de cabeça.

In [20]:
import mlflow
mlflow.set_experiment('house-prices-eda')

INFO: 'house-prices-eda' does not exist. Creating a new experiment


Após executar o comando para configurar o experimento, um novo experimento será criado no MLFlow Tracking

![experiments](../reports/figures/Screenshot_3.png)

## Regressão Linear

In [21]:
mlflow.start_run()

<ActiveRun: >

In [22]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_predicted = lr.predict(X_test)

In [23]:
mlflow.sklearn.log_model(lr, "lr")

Após fazer o ajuste dos modelos, podemos logar o modelo no MLflow, assim o nosso modelo já ficará gravado para futuras reproduções.

![model](../reports/figures/Screenshot_5.png)

Ao logar as métricas do modelo no MLFlow agora podemos rastrear os resultados do nosso modelo na Tracking UI.

In [24]:
mse = mean_squared_error(y_test,lr_predicted)
rmse = math.sqrt(mse)
r2 = r2_score(y_test,lr_predicted)
mlflow.log_metric("mse", mse)
mlflow.log_metric("rmse", rmse)
mlflow.log_metric("r2", r2)

![runs](../reports/figures/Screenshot_4.png)

In [25]:
mlflow.end_run()

## XGBoost

O MLflow já possui diversas integrações com os principais pacotes de Machine Learning, além de possibilitar a utilizar qualquer modelo através da API de Customização já disponível. Vamos criar uma outra execução do modelo, agora utilizando o XGBoost

In [25]:
xgb_params = {
    'learning_rate':0.2,
    'n_estimators':50,
    'random_state':42
}
xgb = XGBRegressor(**xgb_params)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=50, n_jobs=8, num_parallel_tree=1,
             objective='reg:squarederror', random_state=42, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

Ao invés de executar o método start_run() e end_run(), podemos colocar a nossa lógica dentro de um bloco with com apenas o método start_run().

In [26]:
with mlflow.start_run():
    xgb.fit(X_train,y_train)
    mlflow.xgboost.log_model(xgb, "xgboost")
    
    xgb_predicted = xgb.predict(X_test)
    mse = mean_squared_error(y_test,xgb_predicted)
    rmse = math.sqrt(mse)
    r2 = r2_score(y_test,xgb_predicted)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)

Agora temos outra execução do experimento e o podemos comparar de forma simples os resultados de cada uma das técnicas.
![xgboost](../reports/figures/Screenshot_6.png)

## Conhecendo mais o MLflow Tracking

"O componente MLflow Tracking é uma API e IU para registrar parâmetros, versões de código, métricas e arquivos de saída ao executar seu código de aprendizado de máquina e para visualizar posteriormente os resultados. O rastreamento de MLflow permite registrar e consultar experimentos usando Python, REST, R API e APIs Java." MLflow docs

### Onde as informações ficam arquivadas?
"As execuções do MLflow podem ser gravadas em arquivos locais, em um banco de dados compatível com SQLAlchemy ou remotamente em um servidor de rastreamento. Por padrão, os logs da API MLflow Python são executados localmente para arquivos em um diretório 'mlruns' onde quer que você execute seu programa." MLflow docs

Como usamos as configurações padrão do MLflow, todas as informações que estamos utilizando até agora estão sendo armazenadas dentro de uma pasta 'notebooks/mlrun'

![mlrun](../reports/figures/Screenshot_6.png)

### Pesquisando execuções
Todas as informações que visualizamos e armazenamos estão disponíveis para consultarmos programaticamente. E o MLflow tracking permite que essa consulta seja feita usando diversas tecnologias como Python, REST, R API e APIs Java. Vamos avaliar algumas informações usando a API em Python

Apesar de boa parte das informações estarem já disponíveis no Tracking UI, a consulta programatica nos permite recuperar informações para fazer uma análise de dados, usar as informações para alimentar sistemas, dashboards e muito mais.

Através do nome do experimento que criamos podemos recuperar as suas informações.

In [38]:
experiment = mlflow.get_experiment_by_name('house-prices-eda')
experiment

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='house-prices-eda', tags={}>

Com o ID do experimento recuperado acima, podemos listar tdas as execuções feitas.

In [36]:
runs = mlflow.list_run_infos(1)
runs

[<RunInfo: artifact_uri='./mlruns/1/160f68d365de4392b8b95cb651520528/artifacts', end_time=1611857329216, experiment_id='1', lifecycle_stage='active', run_id='160f68d365de4392b8b95cb651520528', run_uuid='160f68d365de4392b8b95cb651520528', start_time=1611857328643, status='FINISHED', user_id='julio'>,
 <RunInfo: artifact_uri='./mlruns/1/e77d2e9d42e243daa5ab5b8e46630cf9/artifacts', end_time=1611857216926, experiment_id='1', lifecycle_stage='active', run_id='e77d2e9d42e243daa5ab5b8e46630cf9', run_uuid='e77d2e9d42e243daa5ab5b8e46630cf9', start_time=1611856938572, status='FINISHED', user_id='julio'>]

Com o ID das execuções podemos listar os modelos e métricas que criamos para em uma específica execucação.

In [37]:
mlflow.get_run('160f68d365de4392b8b95cb651520528')

<Run: data=<RunData: metrics={'mse': 1386727460.1346002,
 'r2': 0.8012741720529797,
 'rmse': 37238.789724353286}, params={}, tags={'mlflow.log-model.history': '[{"run_id": "160f68d365de4392b8b95cb651520528", '
                             '"artifact_path": "xgboost", "utc_time_created": '
                             '"2021-01-28 18:08:48.908708", "flavors": '
                             '{"python_function": {"loader_module": '
                             '"mlflow.xgboost", "python_version": "3.8.5", '
                             '"data": "model.xgb", "env": "conda.yaml"}, '
                             '"xgboost": {"xgb_version": "1.3.3", "data": '
                             '"model.xgb"}}}]',
 'mlflow.source.name': '/home/julio/miniconda3/lib/python3.8/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'julio'}>, info=<RunInfo: artifact_uri='./mlruns/1/160f68d365de4392b8b95cb651520528/artifacts', end_time=1611857329216, experiment_id='1', lifec

Documentação do MLflow tracking https://mlflow.org/docs/latest/tracking.html